In [9]:
import matplotlib
import pandas as pd
from sacrebleu.metrics import BLEU
from typing import Generator, Tuple

In [10]:
SOURCES = "../../data/en-fr/raw/test.fr"
HYPOTHESES = "baseline/translations.txt"
REFERENCES = "../../data/en-fr/raw/test.en"

In [14]:
def ohr_generator() -> Generator[Tuple, None, None]:
    with open(SOURCES, "r") as sources:
        with open(HYPOTHESES, "r") as hypotheses:
            with open(REFERENCES, "r") as references:
                for s, h, r in zip(sources, hypotheses, references):
                    yield s.strip(), h.strip(), r.strip()

shr = [x for x in ohr_generator()]
df = pd.DataFrame(shr, columns=["source", "hypothesis", "reference"])
            

In [15]:
df.head()

,source,hypothesis,reference
0,Linda va être ici.,I &apos;m going to be here for a here .,Linda will be here.
1,Je crains que vous ne soyez descendu au mauvai...,I &apos;m afraid that you don &apos;t be to be...,I'm afraid you got off at the wrong place.
2,Je ferme la porte ?,do I take the door ?,Shall I close the door?
3,Il abuse de son autorité.,he &apos;s trying to his mother of his mother .,He abuses his authority.
4,Il est plus grand que tous les autres garçons.,he is more than all the others we &apos;re tal...,He is taller than any other boy.
